In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
import re

# Manthan's Data

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
base_url = ['https://www.espncricinfo.com/series/ipl-2016-968923/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2017-1078425/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2018-1131611/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2019-1165643/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2020-21-1210595/match-schedule-fixtures-and-results']
soups = []
for s in base_url:
    response = requests.get(s,headers=headers)
    soups.append(BeautifulSoup(response.text,'html.parser'))
    print(response.status_code)

200
200
200
200
200


In [4]:
allseasonmatches = []

for i,soup in enumerate(soups) :
    seasonmatch = []
    anchors = soup.find_all('a',class_='ds-inline-flex ds-items-start ds-leading-none')
    for anchor in anchors :
        link = anchor.get('href')
        link = urljoin(base_url[i],link)
        if 'live-cricket-score' in link :
            seasonmatch.append(link)
    allseasonmatches.append(seasonmatch)
allseasonmatches

[['https://www.espncricinfo.com/series/ipl-2016-968923/mumbai-indians-vs-rising-pune-supergiants-1st-match-980901/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-delhi-daredevils-2nd-match-980903/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-gujarat-lions-3rd-match-980905/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-vs-sunrisers-hyderabad-4th-match-980907/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-mumbai-indians-5th-match-980909/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/gujarat-lions-vs-rising-pune-supergiants-6th-match-980911/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/delhi-daredevils-vs-kings-xi-punjab-7th-match-980913/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers

Columns to fill
1. Match type
2. POTM
3. Venue
4. Team1
5. Team2
6. Toss Winner
7. Toss Decision
8. Winner
9. Win margin
10. Result type(number of wickets left or run difference)
11. Target runs

## Match Details

In [15]:
# asmd - all season match data
# smd - season match data
asmd = []
failed = []
for i,seasonmatch in enumerate(allseasonmatches):
    smd = []
    for j,matchlink in enumerate(seasonmatch):
        try :
            match_data = {
                    'Match Type': '',
                    'POTM': '',
                    'Venue': '',
                    'Team1': '',
                    'Team2': '',
                    'Toss_Winner': '',
                    'Toss_Decision': '',
                    'Winner': '',
                    'Win_Margin': '',
                    'Result_Type': '',
                    'Target_Runs': '',
                    'Season':'',
                    'Match':''
            }

            match_data['Season'] = f"{2016+i}"
            match_data['Match'] = f"{1+j}"
            match = requests.get(matchlink,headers=headers)
            #print(match.status_code)
            soup_match = BeautifulSoup(match.text,'html.parser')
    
            driver = webdriver.Chrome()
            driver.get(matchlink)
    
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
    
            elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
            potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
            teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
            tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
    
            driver.quit()
    
            #  Match type, POTM and Venue
            
            venue = elements.split(',')[1].strip()
            match_data['Venue'] = venue
    
            matching_word = next((word for word in ['Final','Qualifier','Eliminator'] if word in elements), None)
            
            if matching_word:
                match_data['Match Type'] = matching_word
            else:
                match_data['Match Type'] = 'League'
            match_data['POTM'] = potmscrape
            
            # Teams
            
            match_data['Team1'] = teamscrape[0].get_text(strip=True)
            match_data['Team2'] = teamscrape[1].get_text(strip=True)
            
            # Toss Winner and Decision
            
            match_data['Toss_Winner'] = tossscrape.split(',')[0]
            if 'bat' in tossscrape :    
                match_data['Toss_Decision'] = 'bat'
            else :
                match_data['Toss_Decision'] = 'field'
            
            ## Win Details
            
            winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
            match_data['Winner'] = winscrape.split()[0]
            if 'wicket' in winscrape : 
                match_data['Result_Type'] = 'wicket'
            else :
                match_data['Result_Type'] = 'runs'
            
            match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
            
            ## Target Runs
            
            trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
            match_data['Target_Runs'] = trunscrape.split('/')[0]
        except :
            failed.append(matchlink)
        smd.append(match_data)
    asmd.append(smd)
print(failed)

['https://www.espncricinfo.com/series/ipl-2016-968923/mumbai-indians-vs-rising-pune-supergiants-1st-match-980901/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-gujarat-lions-3rd-match-980905/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers-hyderabad-vs-kolkata-knight-riders-8th-match-980915/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-rising-pune-supergiants-10th-match-980919/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/gujarat-lions-vs-sunrisers-hyderabad-15th-match-980929/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/delhi-daredevils-vs-mumbai-indians-17th-match-980933/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers-hyderabad-vs-rising-pune-supergiants-22nd-match-980943/live-cricket-score', 'https://www.espncricinfo.com/series/ipl-2016-968923/rising-pune-supergiants

In [17]:
frames = []
for smd in asmd :
    df = pd.DataFrame(smd)
    frames.append(df)
result = pd.concat(frames)
result

,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,,,,,,,,,,,,2016,1
1,League,Andre Russell,Eden Gardens,DC,KKR,Kolkata Knight Riders,field,KKR,9,wicket,98,2016,2
2,,,,,,,,,,,,2016,3
3,League,AB de Villiers,Bengaluru,RCB,SRH,Sunrisers Hyderabad,field,RCB,45,runs,227,2016,4
4,League,Rohit Sharma,Eden Gardens,KKR,MI,Mumbai Indians,field,MI,6,wicket,187,2016,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,League,Shahbaz Nadeem,Sharjah,MI,SRH,Sunrisers Hyderabad,field,SRH,10,wicket,149,2020,56
56,Qualifier,Jasprit Bumrah,Dubai (DICS),MI,DC,Delhi Capitals,field,MI,57,runs,200,2020,57
57,Eliminator,Kane Williamson,Abu Dhabi,RCB,SRH,Sunrisers Hyderabad,field,SRH,6,wicket,131,2020,58
58,,,,,,,,,,,,2020,59


In [43]:
print(len(failed))
result[result['Season'] == '2018']

80


,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Season,Match
0,,,,,,,,,,,,2018,1
1,League,KL Rahul,Mohali,DC,KXIP,Kings XI Punjab,field,Kings,6,wicket,166,2018,2
2,League,Sunil Narine,Eden Gardens,RCB,KKR,Kolkata Knight Riders,field,KKR,4,wicket,176,2018,3
3,League,Shikhar Dhawan,Hyderabad,RR,SRH,Sunrisers Hyderabad,field,SRH,9,wicket,125,2018,4
4,,,,,,,,,,,,2018,5
5,League,Sanju Samson,Jaipur,RR,DC,Delhi Daredevils,field,RR,10,runs,153,2018,6
6,League,Rashid Khan,Hyderabad,MI,SRH,Sunrisers Hyderabad,field,SRH,1,wicket,147,2018,7
7,League,Umesh Yadav,Bengaluru,KXIP,RCB,Royal Challengers Bangalore,field,RCB,4,wicket,155,2018,8
8,,,,,,,,,,,,2018,9
9,League,Billy Stanlake,Eden Gardens,KKR,SRH,Sunrisers Hyderabad,field,SRH,5,wicket,138,2018,10
